# Domestic Airlines Dataset – Data Cleaning & Preparation

In this notebook, I’m working through the full cleaning and preparation process for a domestic airlines dataset. My goal is to get the data into a clean, reliable format that I can comfortably take into Tableau for visualisation.

I am focusing on:
- Understanding the structure of the dataset  
- Cleaning missing values and fixing data types  
- Creating a proper monthly date column  
- Building useful KPI metrics like on-time rates and delay rates  
- Checking if the data makes sense internally  
- Exporting a final, analysis-ready file  

This gives me a complete end-to-end workflow using Python, which is something I want to get more confident with again.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DomesticAirlinesData.csv to DomesticAirlinesData (1).csv


## 1. Loading and Inspecting the Data

First, I load the dataset into a pandas DataFrame so I can get an idea of what I’m working with.

Here I’m looking at:
- The first few rows  
- Column names and data types  
- How many missing values each column has  

This helps me understand what needs cleaning and what assumptions I’ll have to make.


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('DomesticAirlinesData.csv')
df.head()

,Route,Departing_Port,Arriving_Port,Airline,Month,Sectors_Scheduled,Sectors_Flown,Cancellations,Departures_On_Time,Arrivals_On_Time,Departures_Delayed,Arrivals_Delayed,Year,Month_Num
0,Adelaide-Brisbane,Adelaide,Brisbane,All Airlines,Jan-04,155.0,155,0.0,123.0,120.0,32.0,35.0,2004,1
1,Adelaide-Canberra,Adelaide,Canberra,All Airlines,Jan-04,75.0,75,0.0,72.0,72.0,3.0,3.0,2004,1
2,Adelaide-Gold Coast,Adelaide,Gold Coast,All Airlines,Jan-04,40.0,40,0.0,36.0,35.0,4.0,5.0,2004,1
3,Adelaide-Melbourne,Adelaide,Melbourne,All Airlines,Jan-04,550.0,548,2.0,478.0,487.0,70.0,61.0,2004,1
4,Adelaide-Perth,Adelaide,Perth,All Airlines,Jan-04,191.0,191,0.0,169.0,168.0,22.0,23.0,2004,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114823 entries, 0 to 114822
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Route               114823 non-null  object 
 1   Departing_Port      114823 non-null  object 
 2   Arriving_Port       114823 non-null  object 
 3   Airline             114823 non-null  object 
 4   Month               114823 non-null  object 
 5   Sectors_Scheduled   114823 non-null  float64
 6   Sectors_Flown       114823 non-null  int64  
 7   Cancellations       114507 non-null  float64
 8   Departures_On_Time  114823 non-null  float64
 9   Arrivals_On_Time    114823 non-null  float64
 10  Departures_Delayed  114818 non-null  float64
 11  Arrivals_Delayed    114823 non-null  float64
 12  Year                114823 non-null  int64  
 13  Month_Num           114823 non-null  int64  
dtypes: float64(6), int64(3), object(5)
memory usage: 12.3+ MB


## 2. Handling Missing Values

I noticed that **Cancellations** and **Departures_Delayed** have some missing values.

Since these columns represent event counts, a blank usually means **no event recorded**.  
To keep things consistent (and avoid issues later), I replace missing values in these columns with **0**.

This is a reasonable assumption for operational data and makes the dataset easier to work with.


In [ ]:
df.isna().sum()

,0
Route,0
Departing_Port,0
Arriving_Port,0
Airline,0
Month,0
Sectors_Scheduled,0
Sectors_Flown,0
Cancellations,316
Departures_On_Time,0
Arrivals_On_Time,0


In [ ]:
clean = df.copy()

##3. Splitting the Route Column

The `Route` column contains both the origin and destination in a single string (e.g., "Melbourne-Sydney").
To make analysis and visualisation easier later, I split this column into two new fields:

- **Origin**
- **Destination**

This helps when filtering flights, analysing routes, and building visuals in Tableau.


In [ ]:
# Split Route into Origin and Destination for better analysis
clean[["Origin", "Destination"]] = clean["Route"].str.split("-", expand=True)


In [ ]:
cols_fill_zero = ["Cancellations", "Departures_Delayed"]

for c in cols_fill_zero:
    clean[c] = clean[c].fillna(0)

## 4. Cleaning Data Types

Some columns with flight counts were stored as decimals (like 155.0), even though they should always be whole numbers.

To fix this:
- I round the values just in case  
- Then convert them into integers  

This keeps the data clean and makes calculations more accurate.
# New section

In [ ]:
count_cols = [
    "Sectors_Scheduled",
    "Sectors_Flown",
    "Cancellations",
    "Departures_On_Time",
    "Arrivals_On_Time",
    "Departures_Delayed",
    "Arrivals_Delayed",
]

for c in count_cols:
    clean[c] = clean[c].round().astype(int)


## 5. Creating a Monthly Period Column

The dataset has `Year` and `Month_Num`, but it doesn’t have an actual date column.  
## Since the data is aggregated monthly, I am creating a new `Period` column using the first day of each month.

For example:
- Year: 2004  
- Month_Num: 1  
→ Period becomes 2004-01-01

Using the 1st of the month doesn’t mean flights happened on that day — it just gives Tableau a proper date to work with for time-based charts.


In [ ]:
clean["Period"] = pd.to_datetime({
    "year": clean["Year"],
    "month": clean["Month_Num"],
    "day": 1
})

clean[["Month", "Year", "Month_Num", "Period"]].head()


,Month,Year,Month_Num,Period
0,Jan-04,2004,1,2004-01-01
1,Jan-04,2004,1,2004-01-01
2,Jan-04,2004,1,2004-01-01
3,Jan-04,2004,1,2004-01-01
4,Jan-04,2004,1,2004-01-01


## 6. Creating KPI Columns
Prior to the KPI columns, I have also ra n a code to ensure we have a column repsenting the total number of flights that were supposed and planned to fly.

Then, to make the visualisation more meaningful, I create a few key performance metrics:

### On-Time Rate  
How many flights departed on time out of the flights that actually flew.

### Departure Delay Rate  
Percentage of flights that departed late.

### Arrival Delay Rate  
Percentage of flights that arrived late.

### Cancellation Rate  
Cancelled flights out of total scheduled flights (flown + cancelled).

I also make sure I’m not dividing by zero by checking if the totals are greater than 0 first.

These KPIs will be really helpful later when I build my Tableau dashboard.


In [ ]:
clean["Total_Flights"] = clean["Sectors_Flown"] + clean["Cancellations"]

In [ ]:
# On-Time Rate:
# Number of flights that departed on time / total flights flown
# Checking if Sectors_Flown > 0 to avoid dividing by zero.
clean["On_Time_Rate"] = np.where(
    clean["Sectors_Flown"] > 0,
    clean["Departures_On_Time"] / clean["Sectors_Flown"],
    np.nan
)

In [ ]:
# Departure Delay Rate:
# Number of delayed departures / total flights flown
clean["Departure_Delay_Rate"] = np.where(
    clean["Sectors_Flown"] > 0,
    clean["Departures_Delayed"] / clean["Sectors_Flown"],
    np.nan
)

In [ ]:
# Arrival Delay Rate:
# Number of delayed arrivals / total flights flown
clean["Arrival_Delay_Rate"] = np.where(
    clean["Sectors_Flown"] > 0,
    clean["Arrivals_Delayed"] / clean["Sectors_Flown"],
    np.nan
)

In [ ]:
# Cancellation Rate:
# Cancelled flights / total scheduled flights (flown + cancelled)
clean["Cancellation_Rate"] = np.where(
    clean["Total_Flights"] > 0,
    clean["Cancellations"] / clean["Total_Flights"],
    np.nan
)

## 7. Checking Data Consistency

To double-check that the dataset’s numbers make sense, I verify that:

- On-time departures + delayed departures = flights flown  
- On-time arrivals + delayed arrivals = flights flown  

If they match, I mark the row as `True`.  
If not, it means something is off (some data is missing/wrong).

This step helps confirm that the data is reliable enough for later analysis.


In [ ]:
# Verifying that the number of flights flown is consistent.
# For departures:
# On-time flights + delayed flights = total flights flown (Sectors_Flown)
# If the numbers match → the data is consistent.

clean["Departure_Consistency_OK"] = (
    clean["Departures_On_Time"] + clean["Departures_Delayed"]
) == clean["Sectors_Flown"]

# For arrivals:
# On-time arrivals + delayed arrivals = total flights flown
clean["Arrival_Consistency_OK"] = (
    clean["Arrivals_On_Time"] + clean["Arrivals_Delayed"]
) == clean["Sectors_Flown"]


## 8. Exporting the Cleaned Dataset

Once everything looks good, I export the cleaned data as a new CSV file.  
This version includes:

- Cleaned values  
- Correct data types  
- A proper monthly date column  
- KPI metrics  
- Consistency checks  

This is the file I will use in Tableau for building visualisations and exploring the data further.


In [ ]:
clean.to_csv("DomesticAirlinesData_cleaned.csv", index=False)


In [ ]:
from google.colab import files
files.download("DomesticAirlinesData_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Reviewing Consistency Flags

After creating the consistency checks for departures and arrivals, I wanted to see how many rows were perfectly consistent and how many showed mismatches.

A mismatch here means:
- On-time + delayed departures did *not* equal total flights flown, or
- On-time + delayed arrivals did *not* equal total flights flown.

This type of variation is quite normal in operational datasets, since different airlines or systems may report these metrics separately or with minor discrepancies.

Instead of removing these rows, I keep them in the dataset and use the flags simply to acknowledge areas where reporting might not be perfectly aligned. The overall dataset is still reliable for analysis.


In [ ]:
print("Departure consistency:")
print(clean["Departure_Consistency_OK"].value_counts())

print("\nArrival consistency:")
print(clean["Arrival_Consistency_OK"].value_counts())

Departure consistency:
Departure_Consistency_OK
True     114777
False        46
Name: count, dtype: int64

Arrival consistency:
Arrival_Consistency_OK
True     114771
False        52
Name: count, dtype: int64


### Interpretation

The value counts above show how many rows are fully consistent and how many are not.

Even if a small percentage of rows show `False`, this does not necessarily mean the data is incorrect — it simply highlights slight reporting gaps, which are common in airline operational data.

These inconsistencies do not meaningfully affect the high-level trends I will explore in Tableau, so all rows are kept in the cleaned dataset.


## Link to Dashboard- https://public.tableau.com/views/AustralianDomesticFlightsOn-TimeDelaysCancellations20202025/Dashboard1?:language=en-GB&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link